# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 9:41AM,237825,18,CLRSPC-20977713,"ClearSpec, LLC",Released
1,May 23 2022 9:40AM,237824,10,MET210430,"Methapharm, Inc.",Released
2,May 23 2022 9:40AM,237824,10,MET210432,"Methapharm, Inc.",Released
3,May 23 2022 9:40AM,237824,10,MET210438,"Methapharm, Inc.",Released
4,May 23 2022 9:40AM,237824,10,MET210439,"Methapharm, Inc.",Released
5,May 23 2022 9:40AM,237824,10,MET210442,"Methapharm, Inc.",Released
6,May 23 2022 9:40AM,237822,10,MET210434,"Methapharm, Inc.",Released
7,May 23 2022 9:40AM,237822,10,MET210435,"Methapharm, Inc.",Released
8,May 23 2022 9:40AM,237822,10,MET210436,"Methapharm, Inc.",Released
9,May 23 2022 9:40AM,237822,10,MET210437,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,237819,Released,2
30,237820,Released,20
31,237822,Released,21
32,237824,Released,5
33,237825,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237819,NaN,NaN,2.0
237820,NaN,NaN,20.0
237822,NaN,NaN,21.0
237824,NaN,NaN,5.0
237825,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1.0,24.0,9.0
237781,0.0,0.0,51.0
237787,0.0,0.0,1.0
237789,1.0,6.0,0.0
237792,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1,24,9
237781,0,0,51
237787,0,0,1
237789,1,6,0
237792,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,24,9
1,237781,0,0,51
2,237787,0,0,1
3,237789,1,6,0
4,237792,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,24,9
1,237781,,,51
2,237787,,,1
3,237789,1,6,
4,237792,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC"
1,May 23 2022 9:40AM,237824,10,"Methapharm, Inc."
6,May 23 2022 9:40AM,237822,10,"Methapharm, Inc."
27,May 23 2022 9:37AM,237820,10,"Methapharm, Inc."
47,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc."
49,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc."
52,May 23 2022 9:20AM,237814,10,ISDIN Corporation
80,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC"
83,May 23 2022 9:17AM,237812,10,ISDIN Corporation
133,May 23 2022 9:17AM,237816,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",,,1
1,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",,,5
2,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",,,21
3,May 23 2022 9:37AM,237820,10,"Methapharm, Inc.",,,20
4,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",,,2
5,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",,,3
6,May 23 2022 9:20AM,237814,10,ISDIN Corporation,,,28
7,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",,,3
8,May 23 2022 9:17AM,237812,10,ISDIN Corporation,,,50
9,May 23 2022 9:17AM,237816,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",1,,
1,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",5,,
2,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",21,,
3,May 23 2022 9:37AM,237820,10,"Methapharm, Inc.",20,,
4,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2,,
5,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",3,,
6,May 23 2022 9:20AM,237814,10,ISDIN Corporation,28,,
7,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",3,,
8,May 23 2022 9:17AM,237812,10,ISDIN Corporation,50,,
9,May 23 2022 9:17AM,237816,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",1,,
1,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",5,,
2,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",21,,
3,May 23 2022 9:37AM,237820,10,"Methapharm, Inc.",20,,
4,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",1.0,NaN,NaN
1,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",5.0,NaN,NaN
2,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",21.0,NaN,NaN
3,May 23 2022 9:37AM,237820,10,"Methapharm, Inc.",20.0,NaN,NaN
4,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",1.0,0.0,0.0
1,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",5.0,0.0,0.0
2,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",21.0,0.0,0.0
3,May 23 2022 9:37AM,237820,10,"Methapharm, Inc.",20.0,0.0,0.0
4,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3329333,274.0,24.0,1.0
15,1426797,80.0,0.0,0.0
18,1902436,51.0,0.0,0.0
19,237789,0.0,6.0,1.0
21,237816,1.0,0.0,0.0
22,237787,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3329333,274.0,24.0,1.0
1,15,1426797,80.0,0.0,0.0
2,18,1902436,51.0,0.0,0.0
3,19,237789,0.0,6.0,1.0
4,21,237816,1.0,0.0,0.0
5,22,237787,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,274.0,24.0,1.0
1,15,80.0,0.0,0.0
2,18,51.0,0.0,0.0
3,19,0.0,6.0,1.0
4,21,1.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,274.0
1,15,Released,80.0
2,18,Released,51.0
3,19,Released,0.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,1.0,0.0,0.0
Executing,24.0,0.0,0.0,6.0,0.0,0.0
Released,274.0,80.0,51.0,0.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21,22
0,Completed,1.0,0.0,0.0,1.0,0.0,0.0
1,Executing,24.0,0.0,0.0,6.0,0.0,0.0
2,Released,274.0,80.0,51.0,0.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21,22
0,Completed,1.0,0.0,0.0,1.0,0.0,0.0
1,Executing,24.0,0.0,0.0,6.0,0.0,0.0
2,Released,274.0,80.0,51.0,0.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()